In [ ]:
# From https://www.kaggle.com/bhavikjain/tps-june-21-eda-models

In [2]:
!pip install tensorflow

     |████████████████████████████████| 454.4 MB 56 kB/s  eta 0:00:015   |██▍                             | 33.9 MB 4.2 MB/s eta 0:01:41     |██▋                             | 37.5 MB 4.2 MB/s eta 0:01:40     |███▋                            | 52.0 MB 4.7 MB/s eta 0:01:26     |████▍                           | 61.8 MB 7.5 MB/s eta 0:00:53     |█████▎                          | 74.6 MB 7.3 MB/s eta 0:00:52     |████████                        | 112.4 MB 4.0 MB/s eta 0:01:25     |█████████▉                      | 138.9 MB 3.5 MB/s eta 0:01:30     |███████████▍                    | 161.1 MB 919 kB/s eta 0:05:20     |███████████████▉                | 225.5 MB 4.2 MB/s eta 0:00:55     |████████████████▋               | 235.2 MB 4.8 MB/s eta 0:00:46     |█████████████████               | 241.5 MB 3.6 MB/s eta 0:01:00     |███████████████████             | 271.2 MB 6.4 MB/s eta 0:00:29     |███████████████████▎            | 273.1 MB 6.4 MB/s eta 0:00:29     |████████████████████████▎       | 

      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.0
    Uninstalling typing-extensions-3.10.0.0:
      Successfully uninstalled typing-extensions-3.10.0.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.2.1
    Uninstalling h5py-3.2.1:
      Successfully uninstalled h5py-3.2.1


In [510]:
!pip install tensorflow-addons

     |████████████████████████████████| 679 kB 4.1 MB/s eta 0:00:01


In [517]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss

import gc
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers

from keras.models import Model

In [518]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [519]:
targets = pd.get_dummies(train['target'])

In [520]:
def custom_metric(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-15, 1-1e-15)
    loss = K.mean(cce(y_true, y_pred))
    return loss

cce = tf.keras.losses.CategoricalCrossentropy()

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_custom_metric', min_delta=1e-05, patience=5, verbose=0,
    mode='min', baseline=None, restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_custom_metric', factor=0.7, patience=2, verbose=0,
    mode='min')

In [521]:
def conv_model():

    conv_inputs = layers.Input(shape = (75))
    embed = layers.Embedding (input_dim = 354, 
                              output_dim = 7,
                              embeddings_regularizer='l2')(conv_inputs)
    embed = layers.Conv1D(12,1,activation = 'relu')(embed)        
    embed = layers.Flatten()(embed)
    hidden = layers.Dropout(0.3)(embed)
    
    hidden = tfa.layers.WeightNormalization(
                layers.Dense(
                units=32,
                activation ='selu',
                kernel_initializer = "lecun_normal"))(hidden)
    
    output = layers.Dropout(0.3)(layers.Concatenate()([embed, hidden]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32,
                activation='relu',
                kernel_initializer = "lecun_normal"))(output) 
    output = layers.Dropout(0.4)(layers.Concatenate()([embed, hidden, output]))
    output = tfa.layers.WeightNormalization(
    layers.Dense(
                units = 32, 
                activation = 'relu',
                kernel_initializer = "lecun_normal"))(output)
    
    conv_outputs = layers.Dense(
                units = 9, 
                activation ='softmax',
                kernel_initializer ="lecun_normal")(output)
    
    model = Model(conv_inputs,conv_outputs)
    
    return model

In [ ]:
import datetime;
import time;
print(datetime.datetime.now())
start = time.time()

oof_NN_a = np.zeros((train.shape[0],9))
pred_NN_a = np.zeros((test.shape[0],9))

N_FOLDS = 25
SEED = 2021
EPOCH = 60


skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for fold, (tr_idx, ts_idx) in enumerate(skf.split(train,train.iloc[:,-1])):
    print(f"\n ====== TRAINING FOLD {fold} =======")

    X_train = train.iloc[:,1:-1].iloc[tr_idx]
    y_train = targets.iloc[tr_idx]
    X_test = train.iloc[:,1:-1].iloc[ts_idx]
    y_test = targets.iloc[ts_idx]

    K.clear_session()
    
    #print("\n-----Convolution model Training----\n")

    model_conv = conv_model()

    model_conv.compile(loss='categorical_crossentropy', 
                            optimizer= 'adam', #keras.optimizers.Adam(learning_rate=2e-4), 
                            metrics=custom_metric)
    model_conv.fit(X_train, y_train,
              batch_size = 256, epochs = EPOCH,
              validation_data=(X_test, y_test),
              callbacks=[es, plateau],
              verbose = 0)
   
    pred_a = model_conv.predict(X_test) 
    oof_NN_a[ts_idx] += pred_a 
    score_NN_a = log_loss(y_test, pred_a)
    print(f"\nFOLD {fold} Score convolution model: {score_NN_a}\n")
    pred_NN_a += model_conv.predict(test.iloc[:,1:]) / N_FOLDS 
 
score_a = log_loss(targets, oof_NN_a)
print(f"=== FINAL SCORE CONVOLUTION MODEL : {score_a}===")
print(datetime.datetime.now())
minutes = (time.time() - start) / 60

2021-06-23 13:57:55.184305

 ====== TRAINING FOLD 0 =======


-----Convolution model Training----


FOLD 0 Score convolution model: 1.748694512658287


 ====== TRAINING FOLD 1 =======


-----Convolution model Training----


FOLD 1 Score convolution model: 1.7447382116862573


 ====== TRAINING FOLD 2 =======


-----Convolution model Training----


FOLD 2 Score convolution model: 1.7496628315203124


 ====== TRAINING FOLD 3 =======


-----Convolution model Training----


FOLD 3 Score convolution model: 1.7383812110419385


 ====== TRAINING FOLD 4 =======


-----Convolution model Training----


FOLD 4 Score convolution model: 1.7443089049768168


 ====== TRAINING FOLD 5 =======


-----Convolution model Training----


FOLD 5 Score convolution model: 1.7456763924758416


 ====== TRAINING FOLD 6 =======


-----Convolution model Training----


FOLD 6 Score convolution model: 1.7473318113945424


 ====== TRAINING FOLD 7 =======


-----Convolution model Training----


FOLD 7 Score convolution m

In [508]:
test_probs = model.predict(X_test)
abs(log_loss(y_test, test_probs))

1.9022535900592803

In [314]:
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
test_dummy_y = np_utils.to_categorical(encoded_Y)

test_dummy_y = test_dummy_y.astype(int)
test_y_dim = np.shape(test_dummy_y)[1]

test_probs = grid_search.predict_proba(X_test)
the_log_loss = abs(log_loss(test_dummy_y, test_probs))
print('test log_loss', grid_search.best_score_)
print(the_log_loss)

params = grid_search.best_params_.copy()
params['loss_type'] = loss_type
params['splits_type'] = splits_type
params['n_splits'] = n_splits
params['train_percent'] = str(train_percent * 100) + '%'
params['hidden_nodes'] = hidden_nodes
params['minutes'] = round(minutes, 1)
params['test_log_loss'] = grid_search.best_score_
params['log_loss'] = the_log_loss

#kc_results_df = pd.DataFrame(columns = params)
kc_results_df = kc_results_df.append(params, ignore_index=True)
kc_results_df

test log_loss 0.3508437443524599
1.7748793189344927


,batch_size,epochs,loss_type,splits_type,n_splits,train_percent,hidden_nodes,minutes,log_loss,test_log_loss
0,1000,500,categorical_crossentropy,KFold,8,80.0%,20,12.2,1.767199,NaN
1,1000,500,categorical_crossentropy,KFold,8,80.0%,50,15.1,1.775353,NaN
2,1000,500,categorical_crossentropy,KFold,8,80.0%,35,15.8,1.769479,NaN
3,1000,500,categorical_crossentropy,KFold,8,80.0%,15,11.1,1.770817,NaN
4,500,500,categorical_crossentropy,KFold,8,80.0%,15,16.8,1.767369,NaN
5,256,500,categorical_crossentropy,KFold,8,80.0%,15,25.9,1.765462,NaN
6,256,500,categorical_crossentropy,KFold,8,80.0%,20,27.7,1.766269,NaN
7,500,500,categorical_crossentropy,KFold,8,80.0%,5,16.2,1.777828,NaN
8,500,500,categorical_crossentropy,KFold,25,80.0%,5,47.5,1.777367,NaN
9,500,500,categorical_crossentropy,KFold,25,80.0%,5,47.5,1.777367,0.349412
